In [11]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [54]:
from langchain.document_loaders import TextLoader
loader = TextLoader('docs/FakeWR.txt')
docs = loader.load()
len(docs)
docs[0]

Document(page_content="Certainly, tailoring a Workforce Resilience Program specifically for employees of the Office of Information Technology (OIT) at the Centers for Medicare & Medicaid Services (CMS) means integrating elements pertinent to the nature of their work, the demands of the healthcare sector, and the unique challenges faced by IT professionals. Here's how this could be done:\n\n### CMS OIT Workforce Resilience Program\n\n1. **Program Objective**:\n    - Support the continuous operation of CMS digital infrastructure and health data systems.\n    - Align with healthcare regulations and data protection standards.\n    - Foster a resilient IT community capable of responding to healthcare crises.\n\n2. **Risk Assessment**:\n    - Identify IT-specific challenges, such as cybersecurity threats, system outages, and data breaches.\n    - Evaluate potential health sector crises like pandemics, which may strain digital services.\n\n3. **Training & Education**:\n    - Enhance cybersecu

In [12]:


#web scrape TTB.gov
from bs4 import BeautifulSoup
import requests

docs = []
url = 'https://www.ttb.gov/beer/labeling/malt-beverage-mandatory-label-information'
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Successfully fetched the webpage.")
else:
    print("Failed to fetch the webpage.")

soup = BeautifulSoup(response.text, 'html.parser')
#title = soup.find('title').get_text()
source = soup.find('base').attrs['href']
#docs[0] = soup.getText()


Successfully fetched the webpage.


In [6]:
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader
from bs4 import BeautifulSoup as Soup

#url = "https://docs.python.org/3.9/"
url = "https://js.langchain.com/docs/modules/memory/"
# max_depth=2,extractor=lambda x: Soup(x, "html.parser").text
loader = RecursiveUrlLoader(url=url)
docs = loader.load()

In [7]:
docs

[Document(page_content='\n\n\n\n\nExamples: Memory | 🦜️🔗 Langchain\n\n\n\n\n\nSkip to main content🦜️🔗 LangChainDocsUse casesAPILangSmithPython DocsCTRLKGet startedIntroductionInstallationQuickstartModulesModel I/\u200bORetrievalChainsMemoryHow-toIntegrationsDynamoDB-Backed Chat MemoryFirestore Chat MemoryMomento-Backed Chat MemoryMongoDB Chat MemoryMotörhead MemoryPlanetScale Chat MemoryRedis-Backed Chat MemoryUpstash Redis-Backed Chat MemoryXata Chat MemoryZep MemoryAgentsCallbacksModulesLangChain Expression LanguageCookbookLangChain Expression Language (LCEL)InterfaceGuidesEcosystemAdditional resourcesCommunity navigatorAPI referenceModulesMemoryIntegrationsExamples: Memory📄️ DynamoDB-Backed Chat MemoryFor longer-term persistence across chat sessions, you can swap out the default in-memory chatHistory that backs chat memory classes like BufferMemory for a DynamoDB instance.📄️ Firestore Chat MemoryFor longer-term persistence across chat sessions, you can swap out the default in-memory

In [ ]:
# import
#from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

from langchain.embeddings.openai import OpenAIEmbeddings
import os
from app import get_secrets
os.environ['OPENAI_API_KEY'] = get_secrets(llm='OPENAI')
embedding_function =OpenAIEmbeddings()

from langchain.text_splitter import TokenTextSplitter




# split it into chunks
#text_splitter = CharacterTextSplitter(chunk_size=5000, chunk_overlap=20)
text_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=10)
docs_split = text_splitter.split_documents(docs)
#docs_split=docs




# load it into Chroma
db = Chroma.from_documents(docs_split, embedding_function,persist_directory="./chroma_db")



In [5]:
db

In [ ]:
# query it
query = "What is langchain memory"
docs_q = db.similarity_search(query)

# print results
for d in docs_q:
    print(d.page_content)

In [44]:
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
#from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.chains import LLMChain, ConversationChain
from langchain.chains import RetrievalQA

params = {
    GenParams.MAX_NEW_TOKENS: 1000,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}
credentials = {
    'url': "https://us-south.ml.cloud.ibm.com",
    'apikey' : 
}

project_id = 

flan_t5_model = Model(
model_id="google/flan-t5-xxl",
credentials=credentials,
project_id=project_id)
llm = WatsonxLLM(model=flan_t5_model)


prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
#memory = ConversationBufferMemory(memory_key='chat_history')
#retriever = db.as_retriever()
# Retrieve more documents with higher diversity- useful if your dataset has many similar documents
retriever=db.as_retriever(search_type="mmr", search_kwargs={'k': 10, 'lambda_mult': 0.25})

# Fetch more documents for the MMR algorithm to consider, but only return the top 5
#retriever=db.as_retriever(search_type="mmr", search_kwargs={'k': 5, 'fetch_k': 50})

# Only retrieve documents that have a relevance score above a certain threshold
#retriever= db.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold': 0.7})

# Only get the single most similar document from the dataset
#retriever=db.as_retriever(search_kwargs={'k': 5})

# Use a filter to only retrieve documents from a specific paper 
#docsearch.as_retriever(search_kwargs={'filter': {'paper_title':'GPT-4 Technical Report'}})



qa_s = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever,chain_type_kwargs=chain_type_kwargs)
#chain = LLMChain(llm=llm, prompt=prompt, verbose=True, output_key='output', memory=memory)

In [13]:
qa.run("what is memory")

'memory allows for storing of messages, then later formats the messages into a prompt input variable.'

In [14]:
qa.run("what types of memory")

'Momento-Backed Chat MemoryMongoDB Chat MemoryMotörhead MemoryPlane'

In [15]:
qa.run("how many types of memory")

'There are many types of memory.'

In [16]:
qa.run('What are the types of memory')

'Momento-Backed Chat MemoryMongoDB Chat MemoryMotörhead MemoryPlane'

# With SentenceTransformer

In [31]:
# import
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
#from langchain.document_loaders import TextLoader
import os
#from langchain.text_splitter import TokenTextSplitter

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
docs_split = text_splitter.split_documents(docs)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


# load it into Chroma
db = Chroma.from_documents(docs_split, embedding_function,persist_directory="./chroma_db")
db.persist()


In [46]:
qa_s.run("what is memory")

'Redis-Backed Chat Memory'

In [34]:
qa_s.run("what types of memory")

Number of requested results 50 is greater than number of elements in index 30, updating n_results = 30


'BufferMemory'

In [35]:
qa_s.run("how many types of memory")

Number of requested results 50 is greater than number of elements in index 30, updating n_results = 30


'There are two types of memory.'

In [36]:
qa_s.run('What are the types of memory')

Number of requested results 50 is greater than number of elements in index 30, updating n_results = 30


'Buffer Window Memory |  Langchain Momento-Backed Chat Memory |  Lang'